In [13]:
import pytesseract
from pdf2image import convert_from_path
from tabula import read_pdf
import pandas as pd
import os
import re
import requests
import json


#Method to Extract text from PDF
def extract_text_from_scanned_pdf(pdf_path):
    
    
    images = convert_from_path(pdf_path,poppler_path=r'C:\Program Files\poppler-23.05.0\Library\bin')

    
    extracted_text = []

    # Process each image using OCR
    for image in images:
        # Convert the image to grayscale
        image = image.convert('L')

        # Use pytesseract to extract text from the image
        text = pytesseract.image_to_string(image)

        # Append the extracted text to the list
        extracted_text.append(text)

    # Join the extracted text from all images into a single string
    extracted_text = ' '.join(extracted_text)

    return extracted_text



def convert_scanned_pdf(pdf_path,output_pdf):
    
    # Convert the scanned PDF to images using pdf2image library
    images = convert_from_path(pdf_path, dpi=300)
    # Save the first image from the list
    image_path = "output_image.png"
    images[0].save(image_path, "PNG")
    
    PDF = pytesseract.image_to_pdf_or_hocr(image_path, extension='pdf')

    # export to searchable.pdf
    with open(output_pdf, "w+b") as f:
        f.write(bytearray(PDF))
        




#Function to extract text for header
def extract_text(pattern, text): 
    match = re.search(pattern, text, re.DOTALL | re.MULTILINE)
    if match:
        extracted_text = match.group().strip()
        return extracted_text
    else:
        return "No match found."

    

#Function to extract number for header
def extract_number(pattern,text):
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
    if match:
        extracted_number = match.group(1)
        return extracted_number
    else:
        return "No match found."



#Extract Total
def extract_total(pattern, text):
    match = re.findall(pattern, text)
    if len(match) >= 4:
        extracted_word = match[-4]
        return extracted_word.strip()
    else:
        return "No match found."

#Extract Tax
def extract_tax(pattern, text):
    match = re.findall(pattern, text)
    if len(match) >= 6:
        extracted_word = match[-6]
        return extracted_word.strip()
    else:
        return "No match found."
    

#Extract shipping
def extract_shipping(pattern, text):
    match = re.findall(pattern, text)
    if len(match) >= 7:
        extracted_word = match[-7]
        return extracted_word.strip()
    else:
        return "No match found."

    
#Extract subTotal
def extract_subTotal(pattern, text):
    match = re.findall(pattern, text)
    if len(match) >= 8:
        extracted_word = match[-8]
        return extracted_word.strip()
    else:
        return "No match found."
    
#Extract Currency
def extract_currency(pattern, text):
    match = re.findall(pattern, text)
    if len(match) >= 9:
        extracted_word = match[-9]
        return extracted_word.strip()
    else:
        return "No match found."



#extract items table
def extract_items_data(pdf_file):
    
    area = [960, 0, 10000, 5000]  # [top, left, bottom, right]
    columns = [295, 1720, 1990, 2318, 2580, 2825, 3660]
    
    # Read the PDF and extract the table(s)
    table_pdf = read_pdf(pdf_file, multiple_tables=True, guess=False, pages='all', lattice=False,
                         stream=True, encoding="utf-8", area=area, columns=columns, silent=True, output_format="dataframe")

    
    # Create empty lists to store the extracted data
    ordered_column = []
    shipped_column = []
    unitPrice_column = []
    Extended_Amount_column = []
    description_column = []

    
    # Iterate over the tables
    for i in range(len(table_pdf)):
        # Get the current table
        table = table_pdf[i]

        # Find the index of the row that meets the specified condition
        condition_row_index = table[
            (table.iloc[:, 3] == "SHIPPING/HANDL") &
            (table.iloc[:, 4] == "ING TA") &
            (table.iloc[:, 5] == "X") &
            (table.iloc[:, 6] == "TOTAL")
        ].index

        
        # If the condition is met in the table
        if not condition_row_index.empty:
            
            condition_row = condition_row_index[0]
            table = table.iloc[:condition_row + 1, :]

        
     
        table = table.iloc[1:-1, 1:]
        
        #exceptional condition
        #table = table.iloc[:-1, 1:]
        
        
        # Remove rows with only null values
        table = table.dropna(how='all')

        
        # Store non-null values of column two in the list
        ordered_column.extend(table.iloc[:, 1].dropna().tolist())
        shipped_column.extend(table.iloc[:, 2].dropna().tolist())
        unitPrice_column.extend(table.iloc[:, 3].dropna().tolist())
        Extended_Amount_column.extend(table.iloc[:, 5].dropna().tolist())

        
        # Extract data from column one as a string
        column_one_data = " ".join(table.iloc[:, 0].dropna().tolist())

        # Split the extracted data by the eight-digit number
        processed_data = re.split(r'(\d{8})', column_one_data)

        # Remove empty strings from the processed data
        processed_data = [item.strip() for item in processed_data if item.strip()]

        
        # Modify the processed data
        for j in range(1, len(processed_data), 2):
            processed_data[j - 1] += processed_data[j]

        # Remove the elements at odd indices
        processed_data = processed_data[::2]

        # Store the processed data in the list
        description_column.extend(processed_data)

        # Assign the modified table back to the list
        table_pdf[i] = table


    # Convert the shipped_column values to strings and replace '.' with ','
    shipped_column = [str(value).replace('.', ',') for value in shipped_column]
    

    # Convert the ordered_column values to strings and replace '.' with ','
    ordered_column = [str(value).replace('.', ',') for value in ordered_column]
    
    
    # Convert the ordered_column values to strings and replace '.' with ','
    Extended_Amount_column = [str(value).replace(' ', '.') for value in Extended_Amount_column]
    
    #write data into json format
    json_data = {"items": []}
    for i in range(len(ordered_column)):
        data = {
            'No' : i+1,
            'Description':description_column[i],
            'Ordered': ordered_column[i],
            'Shipped': shipped_column[i],
            'unitPrice': unitPrice_column[i],
            'Tax' : "",
            'Extended_Amount': Extended_Amount_column[i],
        }
        json_data["items"].append(data)
    return json_data


    


    
#Patterns to extract data of Header and Footer
customer_pattern = r"(?i)\b[A-Z\s-]+\bSDN\. BHD\."
PO_pattern = r"\bPO\d{6}\b"
sales_order_pattern =  r"SALES ORDER NUMBER.*?(\d{7})"
packing_pattern = r"PACKING SLIP #.*?\d{7}.*?(\d{7})"
customer_number_pattern =  r"CUSTOMER NUMBER.*?(\d{5})"
total_pattern = r"\b([\w.,']+)\b"
pattern = r"\b([\w.,']+)\b"
number_pattern = r'\b\d{5}\b'





#josn objects to store data
header_data = {}
footer_data = {}
items_data = {}




# Prompt user for file path
pdf_file = input("Enter the file path: ")




# Specify the path to your scanned PDF
#pdf_file = 'INV55.pdf'


#write name of your pure pdf output
DestinationFile = "pure_pdf_.pdf"

#write destination json file name
Destination_Json = pdf_file.replace(".pdf", ".json")



# Extract text from the scanned PDF
text = extract_text_from_scanned_pdf(pdf_file)



# #convert scanned pdf to pure pdf
# convert_to_searchable_pdf(API_KEY, pdf_file, DestinationFile)

convert_scanned_pdf(pdf_file, DestinationFile)


print("------------------------- HEADER -------------------------- ")

# Doctype
Doctype = "Invoice"
header_data['Doctype'] = Doctype
print("Doctype:", Doctype)

# Customer
Customer = extract_text(customer_pattern, text)
header_data['Customer'] = Customer
print("Customer:", Customer)


number = extract_text(number_pattern,pdf_file)
header_data['Number'] = number
print("Number:", number)

# PO
PO = extract_text(PO_pattern, text)
header_data['PO'] = PO
print("PO:", PO)


# Sales_Order_number
SalesOrder = extract_number(sales_order_pattern, text)
header_data['SalesOrder'] = SalesOrder
print("SalesOrder:", SalesOrder)


# Packing Slip
Packing = extract_number(packing_pattern, text)
header_data['Packing'] = Packing
print("Packing:", Packing)


# Customer Number
customer_number = extract_number(customer_number_pattern, text)
header_data['CustomerNumber'] = customer_number
print("CustomerNumber:", customer_number)


print("------------------------- ITEMS -------------------------- ")


# Create a JSON object
items_data = extract_items_data(DestinationFile)

print(items_data)


print("------------------------- FOOTER -------------------------- ")


# Currency
extracted_currency = extract_currency(pattern, text)
footer_data['Currency'] = extracted_currency
print("Currency:", extracted_currency)


# subTotal
extracted_subTotal = extract_subTotal(pattern, text)
footer_data['SubTotal'] = extracted_subTotal
print("SubTotal:", extracted_subTotal)


# shipping
extracted_shipping = extract_shipping(pattern, text)
footer_data['Shipping'] = extracted_shipping
print("Shipping:", extracted_shipping)


# Tax
extracted_tax = extract_tax(pattern, text)
footer_data['Tax'] = extracted_tax
print("Tax:", extracted_tax)


# Total
extracted_total = extract_total(pattern, text)
footer_data['Total'] = extracted_total
print("Total:", extracted_total)




print("---------------- WRITE DATA INTO JSON FORMAT -----------------------------")

# Create a new JSON file and write the data
with open(Destination_Json, 'w') as file:
    json.dump({'Header': header_data,'Items' : items_data , 'Footer': footer_data}, file, indent=4)

print("Generate Json file")    
print(f"Data written to {Destination_Json} file.")



Enter the file path: F:\Music\8th\invoice scrapping\final_code\INV 92481.pdf
------------------------- HEADER -------------------------- 
Doctype: Invoice
Customer: COHERENT MALAYSIA SDN. BHD.
Number: 92481
PO: PO780239
SalesOrder: 2033962
Packing: 1222305
CustomerNumber: 32639
------------------------- ITEMS -------------------------- 
{'items': [{'No': 1, 'Description': "1207728 ASY,TOSA,6PIN,BALL LENS,10G DFB,INTEGRATED DRIVER,LC RECEPTACLE ISOLATOR,GEN 2,(-5C,RT,75C), PO_LINE_NUMBER: , S/N's : , Commercial Invoice :14161214", 'Ordered': '247', 'Shipped': '247', 'unitPrice': '16.27726', 'Tax': '', 'Extended_Amount': '4,020.48'}, {'No': 2, 'Description': "1207728 ASY,TOSA,6PIN,BALL LENS,10G DFB,INTEGRATED DRIVER,LC RECEPTACLE ISOLATOR,GEN 2,(-5C,RT,75C), PO_LINE_NUMBER: , S/N's : , Commercial Invoice :14161215", 'Ordered': '267', 'Shipped': '267', 'unitPrice': '16.27726', 'Tax': '', 'Extended_Amount': '4,346.03'}]}
------------------------- FOOTER -------------------------- 
Currency